# LAB | Ensemble Methods

**Load the data**

In this challenge, we will be working with the same Spaceship Titanic data, like the previous Lab. The data can be found here:

https://raw.githubusercontent.com/data-bootcamp-v4/data/main/spaceship_titanic.csv

Metadata

https://github.com/data-bootcamp-v4/data/blob/main/spaceship_titanic.md

In this Lab, you should try different ensemble methods in order to see if can obtain a better model than before. In order to do a fair comparison, you should perform the same feature scaling, engineering applied in previous Lab.

In [1]:
#Libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LassoCV
from sklearn.ensemble import BaggingClassifier, VotingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import AdaBoostClassifier


In [2]:
spaceship = pd.read_csv("https://raw.githubusercontent.com/data-bootcamp-v4/data/main/spaceship_titanic.csv")
spaceship.head()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True


Now perform the same as before:
- Feature Scaling
- Feature Selection


In [3]:

numeric_features = ['Age', 'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']

scaler = StandardScaler()
spaceship[numeric_features] = scaler.fit_transform(spaceship[numeric_features])

spaceship.dropna(inplace=True)
results = {}
spaceship.head()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,0.702095,False,-0.337025,-0.284274,-0.287317,-0.273736,-0.266098,Maham Ofracculy,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,-0.333233,False,-0.173528,-0.278689,-0.245971,0.209267,-0.227692,Juanna Vines,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,2.013510,True,-0.272527,1.934922,-0.287317,5.634034,-0.223327,Altark Susent,False
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,0.287964,False,-0.337025,0.511931,0.326250,2.655075,-0.097634,Solam Susent,False
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,-0.885407,False,0.117466,-0.240833,-0.037590,0.223344,-0.264352,Willy Santantines,True


In [4]:

X = spaceship[numeric_features + ['HomePlanet', 'CryoSleep', 'Destination', 'VIP']]
y = spaceship['Transported']

X = pd.get_dummies(X, drop_first=True)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

lasso = LassoCV(cv=5)

lasso.fit(X_train, y_train)

importance = pd.Series(lasso.coef_, index=X.columns)
print(importance[importance != 0])


Age                         -0.033930
RoomService                 -0.081082
FoodCourt                    0.068493
ShoppingMall                 0.051534
Spa                         -0.101864
VRDeck                      -0.092589
HomePlanet_Europa            0.249882
HomePlanet_Mars              0.114377
CryoSleep_True               0.377535
Destination_PSO J318.5-22   -0.063736
Destination_TRAPPIST-1e     -0.078550
VIP_True                    -0.096494
dtype: float64


**Perform Train Test Split**

In [5]:
#your code here
X = spaceship[numeric_features + ['HomePlanet', 'CryoSleep', 'Destination', 'VIP']]
y = spaceship['Transported']

X = pd.get_dummies(X, drop_first=True)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(f"Tamaño del conjunto de entrenamiento: {X_train.shape[0]} muestras")
print(f"Tamaño del conjunto de prueba: {X_test.shape[0]} muestras")

Tamaño del conjunto de entrenamiento: 5284 muestras
Tamaño del conjunto de prueba: 1322 muestras


**Model Selection** - now you will try to apply different ensemble methods in order to get a better model

- Bagging and Pasting

In [6]:
#your code here
X = spaceship[numeric_features + ['HomePlanet', 'CryoSleep', 'Destination', 'VIP']]
y = spaceship['Transported']
X = pd.get_dummies(X, drop_first=True)

# Dividir el conjunto de datos
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Bagging con DecisionTreeClassifier
bagging_model = BaggingClassifier(estimator=DecisionTreeClassifier(), 
                                   n_estimators=100, 
                                   random_state=42)

# Entrenar el modelo
bagging_model.fit(X_train, y_train)

# Hacer predicciones
y_pred_bagging = bagging_model.predict(X_test)

# Evaluar el modelo
bagging_accuracy = accuracy_score(y_test, y_pred_bagging)
print(f'Accuracy del modelo de Bagging: {bagging_accuracy:.2f}')

# Pasting con DecisionTreeClassifier
pasting_model = BaggingClassifier(estimator=DecisionTreeClassifier(), 
                                   n_estimators=100, 
                                   bootstrap=False,  # Sin reemplazo
                                   random_state=42)

# Entrenar el modelo
pasting_model.fit(X_train, y_train)

# Hacer predicciones
y_pred_pasting = pasting_model.predict(X_test)

# Evaluar el modelo
pasting_accuracy = accuracy_score(y_test, y_pred_pasting)
results['Bagging and Pasting'] = pasting_accuracy
print(f'Accuracy del modelo de Pasting: {pasting_accuracy:.2f}')

Accuracy del modelo de Bagging: 0.80
Accuracy del modelo de Pasting: 0.75


- Random Forests

In [7]:
#your code here
X = spaceship[numeric_features + ['HomePlanet', 'CryoSleep', 'Destination', 'VIP']]
y = spaceship['Transported']
X = pd.get_dummies(X, drop_first=True)

# Dividir el conjunto de datos
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Crear el modelo de Random Forest
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)

# Entrenar el modelo
rf_model.fit(X_train, y_train)

# Hacer predicciones
y_pred_rf = rf_model.predict(X_test)

# Evaluar el modelo
rf_accuracy = accuracy_score(y_test, y_pred_rf)
results['Random Forest'] = rf_accuracy
print(f'Accuracy del modelo de Random Forest: {rf_accuracy:.2f}')

Accuracy del modelo de Random Forest: 0.80


- Gradient Boosting

In [8]:
#your code here
X = spaceship[numeric_features + ['HomePlanet', 'CryoSleep', 'Destination', 'VIP']]
y = spaceship['Transported']
X = pd.get_dummies(X, drop_first=True)

# Dividir el conjunto de datos
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Crear el modelo de Gradient Boosting
gb_model = GradientBoostingClassifier(n_estimators=100, random_state=42)

# Entrenar el modelo
gb_model.fit(X_train, y_train)

# Hacer predicciones
y_pred_gb = gb_model.predict(X_test)

# Evaluar el modelo
gb_accuracy = accuracy_score(y_test, y_pred_gb)
results['Gradient Boosting'] = gb_accuracy
print(f'Accuracy del modelo de Gradient Boosting: {gb_accuracy:.2f}')

Accuracy del modelo de Gradient Boosting: 0.80


- Adaptive Boosting

In [9]:
#your code here
X = spaceship[numeric_features + ['HomePlanet', 'CryoSleep', 'Destination', 'VIP']]
y = spaceship['Transported']
X = pd.get_dummies(X, drop_first=True)

# Dividir el conjunto de datos
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Crear el modelo de AdaBoost
# Usaremos un árbol de decisión como clasificador base
ada_model = AdaBoostClassifier(estimator=DecisionTreeClassifier(max_depth=1), 
                                n_estimators=100, 
                                random_state=42)

# Entrenar el modelo
ada_model.fit(X_train, y_train)

# Hacer predicciones
y_pred_ada = ada_model.predict(X_test)

# Evaluar el modelo
ada_accuracy = accuracy_score(y_test, y_pred_ada)
results["Adaptative Boosting"] = ada_accuracy
print(f'Accuracy del modelo de AdaBoost: {ada_accuracy:.2f}')

Accuracy del modelo de AdaBoost: 0.79


/opt/anaconda3/envs/env_lab/lib/python3.11/site-packages/sklearn/ensemble/_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(


Which model is the best and why?

In [10]:
results_df = pd.DataFrame(results.items(), columns=['Modelo', 'Precisión'])
results_df.sort_values(by='Precisión', ascending=False, inplace=True)

# Mostrar los resultados
print("Resultados de los Modelos:")
print(results_df)

# Determinar y mostrar el mejor modelo
best_model = results_df.iloc[0]
print(f"\nEl mejor modelo es: {best_model['Modelo']} con una precisión de: {best_model['Precisión']:.2f}")


Resultados de los Modelos:
                Modelo  Precisión
2    Gradient Boosting   0.798790
1        Random Forest   0.796520
3  Adaptative Boosting   0.789713
0  Bagging and Pasting   0.747352

El mejor modelo es: Gradient Boosting con una precisión de: 0.80
